In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
%run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [4]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
syllable_dictionary = create_syllables_dictionary(content, vowels)
max_syllable = get_max_syllable(syllable_dictionary)
syllable_letters_translator = create_syllable_letters_translator(max_syllable, syllable_dictionary, dictionary, vowels)

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!


In [64]:
%run ../../../prepare_data.py
gen = generate_fake_epoch_syllabled_letters(X_train[:20], X_other_features_train[:20], y_train[:20], 16, syllable_letters_translator)

In [66]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
print(np.array(test[0][0]).shape)
for el in test[0][0][0]:
    print(decode_input(el, dictionary))
    print(el)
    
#     print(syllable_dictionary[el.tolist().index(1)])

(16, 11, 210)


TypeError: 'numpy.int64' object is not iterable

In [65]:
# print(X_train[0])
test = next(gen)
# eye = np.eye(5168)[X_train[0]]
# print(X_train[0].tolist())
# np.eye(5168)[[2, 1]]



In [123]:
np.eye(10)[[3,7,2,5]]
print(eye[[2, 3]].shape)

(2, 5168)


In [11]:
X_validate.shape
# len(syllable_dictionary)

(54222, 11)

In [5]:
print(len(syllable_dictionary))
shuffle_vector_location = '../../internal_representations/inputs/shuffle_vector'
X_train, X_other_features_train, y_train = generate_syllable_X_and_y(syllable_dictionary, max_word, max_num_vowels, train_content, vowels, accetuated_vowels, feature_dictionary, shuffle_vector_location + '_train.h5')


5168
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!


In [13]:
%run ../../../prepare_data.py
# print(syllable_dictionary)
syllable_letters_translator = create_syllable_letters_translator(max_syllable, syllable_dictionary, dictionary, vowels)
print(syllable_letters_translator[3])
print(syllable_dictionary)
#     [[[1 if let < len(el) and a == el[let] else 0 for a in dictionary] for let in range(max_syllable)] for el in syllable_dictionary]

# [[[1 if let < len(el) and a == el[let] else 0 for a in dictionary] for let in range(max_syllable)] for el in syllable_dictionary]

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['', '-o', 'a', 'ab', 'ah', 'akt', 'al', 'alf', 'alg', 'alt', 'am', 'an', 'ant', 'ar', 'arh', 'as', 'at', 'av', 'ba', 'bab', 'bah', 'baj', 'bajt', 'bak', 'bal', 'bam', 'ban', 'banj', 'bank', 'bar', 'bark', 'barv', 'bas', 'bast', 'bat', 'bav', 'baz', 'bač', 'baš', 'baž', 'bde', 'bdel', 'bdet', 'bdeč', 'bdi', 'bdim', 'bdiš', 'be', 'bec', 'bed', 'beg', 'beh', 'bej', 'bek', 'bel', 'belc', 'belj', 'belk', 'bem', 'ben', 'benj', 'benk', 'bent', 'ber', 'bes', 'bet', 'bev', 'bez', 'beč', 'beš

In [7]:
print (X_train[0])
print (X_train.shape)
print (X_test.shape)
print (X_validate.shape)
# X_other_features = create_X_features(content)
# print (X_other_features[178200])
decode_position = 38
print (decode_input(X_validate[decode_position], dictionary))
# print (X_other_features_train[0])
print (decode_X_features(feature_dictionary, [X_other_features_train[decode_position]]))
# print (len(X_other_features_train[0]))

[2130 1226 1677    2 1873 4738    0    0    0    0    0]
(430151, 11)


NameError: name 'X_test' is not defined

In [10]:
num_examples = len(X_train) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 11
nn_hdim = 516
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [16]:
conv_input_shape=(11, 252)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(44, (3), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)

In [17]:
model = Model(inputs=[conv_input, othr_input], outputs=x)

# epochs = 5
# lrate = 0.1
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile model
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [18]:
history = model.fit_generator(generate_fake_epoch_syllabled_letters(X_train, X_other_features_train, y_train, batch_size, syllable_letters_translator), X_train.shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=generate_fake_epoch_syllabled_letters(X_test, X_other_features_test, y_test, batch_size, syllable_letters_translator), validation_steps=X_test.shape[0]/(batch_size * num_fake_epoch))

Epoch 1/800
1345/1344 [==============================] - 23s - loss: 0.1818 - actual_accuracy: 0.3763 - val_loss: 0.1201 - val_actual_accuracy: 0.6185
Epoch 2/800
1345/1344 [==============================] - 31s - loss: 0.1113 - actual_accuracy: 0.6371 - val_loss: 0.1010 - val_actual_accuracy: 0.7109
Epoch 3/800
1345/1344 [==============================] - 28s - loss: 0.0978 - actual_accuracy: 0.7060 - val_loss: 0.0943 - val_actual_accuracy: 0.7469
Epoch 4/800
1345/1344 [==============================] - 30s - loss: 0.0904 - actual_accuracy: 0.7401 - val_loss: 0.0883 - val_actual_accuracy: 0.7738
Epoch 5/800
1345/1344 [==============================] - 28s - loss: 0.0849 - actual_accuracy: 0.7608 - val_loss: 0.0906 - val_actual_accuracy: 0.7577
Epoch 6/800
1345/1344 [==============================] - 31s - loss: 0.0792 - actual_accuracy: 0.7783 - val_loss: 0.0849 - val_actual_accuracy: 0.7826
Epoch 7/800
1345/1344 [==============================] - 33s - loss: 0.0752 - actual_accuracy:

1345/1344 [==============================] - 30s - loss: 0.0229 - actual_accuracy: 0.9430 - val_loss: 0.1099 - val_actual_accuracy: 0.8505
Epoch 55/800
1345/1344 [==============================] - 31s - loss: 0.0228 - actual_accuracy: 0.9431 - val_loss: 0.1097 - val_actual_accuracy: 0.8516
Epoch 56/800
1345/1344 [==============================] - 29s - loss: 0.0225 - actual_accuracy: 0.9452 - val_loss: 0.1075 - val_actual_accuracy: 0.8577
Epoch 57/800
1345/1344 [==============================] - 33s - loss: 0.0215 - actual_accuracy: 0.9495 - val_loss: 0.1047 - val_actual_accuracy: 0.8616
Epoch 58/800
1345/1344 [==============================] - 27s - loss: 0.0213 - actual_accuracy: 0.9480 - val_loss: 0.1171 - val_actual_accuracy: 0.8467
Epoch 59/800
1345/1344 [==============================] - 28s - loss: 0.0218 - actual_accuracy: 0.9461 - val_loss: 0.1131 - val_actual_accuracy: 0.8370
Epoch 60/800
1345/1344 [==============================] - 32s - loss: 0.0213 - actual_accuracy: 0.947

1345/1344 [==============================] - 35s - loss: 0.0143 - actual_accuracy: 0.9660 - val_loss: 0.1440 - val_actual_accuracy: 0.8413
Epoch 108/800
1345/1344 [==============================] - 30s - loss: 0.0150 - actual_accuracy: 0.9634 - val_loss: 0.1143 - val_actual_accuracy: 0.8620
Epoch 109/800
1345/1344 [==============================] - 34s - loss: 0.0137 - actual_accuracy: 0.9667 - val_loss: 0.1260 - val_actual_accuracy: 0.8570
Epoch 110/800
1345/1344 [==============================] - 27s - loss: 0.0147 - actual_accuracy: 0.9642 - val_loss: 0.1360 - val_actual_accuracy: 0.8597
Epoch 111/800
1345/1344 [==============================] - 31s - loss: 0.0139 - actual_accuracy: 0.9671 - val_loss: 0.1308 - val_actual_accuracy: 0.8520
Epoch 112/800
1345/1344 [==============================] - 28s - loss: 0.0140 - actual_accuracy: 0.9665 - val_loss: 0.1191 - val_actual_accuracy: 0.8589
Epoch 113/800
1345/1344 [==============================] - 26s - loss: 0.0132 - actual_accuracy:

1345/1344 [==============================] - 33s - loss: 0.0114 - actual_accuracy: 0.9719 - val_loss: 0.1421 - val_actual_accuracy: 0.8631
Epoch 161/800
1345/1344 [==============================] - 29s - loss: 0.0107 - actual_accuracy: 0.9748 - val_loss: 0.1430 - val_actual_accuracy: 0.8600
Epoch 162/800
1345/1344 [==============================] - 35s - loss: 0.0113 - actual_accuracy: 0.9731 - val_loss: 0.1383 - val_actual_accuracy: 0.8654
Epoch 163/800
1345/1344 [==============================] - 28s - loss: 0.0106 - actual_accuracy: 0.9745 - val_loss: 0.1204 - val_actual_accuracy: 0.8616
Epoch 164/800
1345/1344 [==============================] - 34s - loss: 0.0113 - actual_accuracy: 0.9725 - val_loss: 0.1283 - val_actual_accuracy: 0.8581
Epoch 165/800
1345/1344 [==============================] - 33s - loss: 0.0109 - actual_accuracy: 0.9750 - val_loss: 0.1423 - val_actual_accuracy: 0.8455
Epoch 166/800
1345/1344 [==============================] - 33s - loss: 0.0111 - actual_accuracy:

1345/1344 [==============================] - 36s - loss: 0.0088 - actual_accuracy: 0.9794 - val_loss: 0.1528 - val_actual_accuracy: 0.8620
Epoch 214/800
1345/1344 [==============================] - 33s - loss: 0.0093 - actual_accuracy: 0.9782 - val_loss: 0.1609 - val_actual_accuracy: 0.8551
Epoch 215/800
1345/1344 [==============================] - 31s - loss: 0.0091 - actual_accuracy: 0.9787 - val_loss: 0.1522 - val_actual_accuracy: 0.8643
Epoch 216/800
1345/1344 [==============================] - 31s - loss: 0.0089 - actual_accuracy: 0.9797 - val_loss: 0.1480 - val_actual_accuracy: 0.8712
Epoch 217/800
1345/1344 [==============================] - 30s - loss: 0.0090 - actual_accuracy: 0.9787 - val_loss: 0.1510 - val_actual_accuracy: 0.8577
Epoch 218/800
1345/1344 [==============================] - 33s - loss: 0.0089 - actual_accuracy: 0.9796 - val_loss: 0.1553 - val_actual_accuracy: 0.8627
Epoch 219/800
1345/1344 [==============================] - 32s - loss: 0.0091 - actual_accuracy:

...

1345/1344 [==============================] - 37s - loss: 0.0053 - actual_accuracy: 0.9882 - val_loss: 0.1670 - val_actual_accuracy: 0.8692
Epoch 585/800
1345/1344 [==============================] - 34s - loss: 0.0050 - actual_accuracy: 0.9881 - val_loss: 0.1761 - val_actual_accuracy: 0.8539
Epoch 586/800
1345/1344 [==============================] - 35s - loss: 0.0051 - actual_accuracy: 0.9887 - val_loss: 0.1754 - val_actual_accuracy: 0.8631
Epoch 587/800
1345/1344 [==============================] - 38s - loss: 0.0051 - actual_accuracy: 0.9885 - val_loss: 0.1520 - val_actual_accuracy: 0.8685
Epoch 588/800
1345/1344 [==============================] - 37s - loss: 0.0046 - actual_accuracy: 0.9885 - val_loss: 0.1588 - val_actual_accuracy: 0.8773
Epoch 589/800
1345/1344 [==============================] - 43s - loss: 0.0051 - actual_accuracy: 0.9888 - val_loss: 0.1660 - val_actual_accuracy: 0.8635
Epoch 590/800
1345/1344 [==============================] - 36s - loss: 0.0052 - actual_accuracy:

1345/1344 [==============================] - 35s - loss: 0.0051 - actual_accuracy: 0.9882 - val_loss: 0.1623 - val_actual_accuracy: 0.8635
Epoch 638/800
1345/1344 [==============================] - 35s - loss: 0.0051 - actual_accuracy: 0.9882 - val_loss: 0.1646 - val_actual_accuracy: 0.8735
Epoch 639/800
1345/1344 [==============================] - 33s - loss: 0.0053 - actual_accuracy: 0.9882 - val_loss: 0.1687 - val_actual_accuracy: 0.8735
Epoch 640/800
1345/1344 [==============================] - 36s - loss: 0.0056 - actual_accuracy: 0.9885 - val_loss: 0.1628 - val_actual_accuracy: 0.8600
Epoch 641/800
1345/1344 [==============================] - 37s - loss: 0.0052 - actual_accuracy: 0.9882 - val_loss: 0.1695 - val_actual_accuracy: 0.8627
Epoch 642/800
1345/1344 [==============================] - 32s - loss: 0.0051 - actual_accuracy: 0.9888 - val_loss: 0.1533 - val_actual_accuracy: 0.8823
Epoch 643/800
1345/1344 [==============================] - 35s - loss: 0.0045 - actual_accuracy:

1345/1344 [==============================] - 40s - loss: 0.0043 - actual_accuracy: 0.9902 - val_loss: 0.1553 - val_actual_accuracy: 0.8792
Epoch 691/800
1345/1344 [==============================] - 34s - loss: 0.0052 - actual_accuracy: 0.9884 - val_loss: 0.1833 - val_actual_accuracy: 0.8604
Epoch 692/800
1345/1344 [==============================] - 34s - loss: 0.0048 - actual_accuracy: 0.9892 - val_loss: 0.1628 - val_actual_accuracy: 0.8738
Epoch 693/800
1345/1344 [==============================] - 41s - loss: 0.0048 - actual_accuracy: 0.9895 - val_loss: 0.1616 - val_actual_accuracy: 0.8658
Epoch 694/800
1345/1344 [==============================] - 36s - loss: 0.0046 - actual_accuracy: 0.9893 - val_loss: 0.1523 - val_actual_accuracy: 0.8643
Epoch 695/800
1345/1344 [==============================] - 36s - loss: 0.0048 - actual_accuracy: 0.9881 - val_loss: 0.1716 - val_actual_accuracy: 0.8669
Epoch 696/800
1345/1344 [==============================] - 38s - loss: 0.0042 - actual_accuracy:

1345/1344 [==============================] - 36s - loss: 0.0041 - actual_accuracy: 0.9909 - val_loss: 0.1665 - val_actual_accuracy: 0.8804
Epoch 744/800
1345/1344 [==============================] - 40s - loss: 0.0049 - actual_accuracy: 0.9896 - val_loss: 0.1764 - val_actual_accuracy: 0.8547
Epoch 745/800
1345/1344 [==============================] - 42s - loss: 0.0051 - actual_accuracy: 0.9879 - val_loss: 0.1726 - val_actual_accuracy: 0.8585
Epoch 746/800
1345/1344 [==============================] - 40s - loss: 0.0041 - actual_accuracy: 0.9904 - val_loss: 0.1828 - val_actual_accuracy: 0.8539
Epoch 747/800
1345/1344 [==============================] - 44s - loss: 0.0051 - actual_accuracy: 0.9883 - val_loss: 0.1636 - val_actual_accuracy: 0.8639
Epoch 748/800
1345/1344 [==============================] - 39s - loss: 0.0045 - actual_accuracy: 0.9895 - val_loss: 0.1729 - val_actual_accuracy: 0.8712
Epoch 749/800
1345/1344 [==============================] - 35s - loss: 0.0046 - actual_accuracy:

1345/1344 [==============================] - 37s - loss: 0.0046 - actual_accuracy: 0.9894 - val_loss: 0.1810 - val_actual_accuracy: 0.8620
Epoch 797/800
1345/1344 [==============================] - 37s - loss: 0.0040 - actual_accuracy: 0.9913 - val_loss: 0.1629 - val_actual_accuracy: 0.8823
Epoch 798/800
1345/1344 [==============================] - 46s - loss: 0.0045 - actual_accuracy: 0.9898 - val_loss: 0.1869 - val_actual_accuracy: 0.8577
Epoch 799/800
1345/1344 [==============================] - 48s - loss: 0.0051 - actual_accuracy: 0.9890 - val_loss: 0.1940 - val_actual_accuracy: 0.8616
Epoch 800/800
1345/1344 [==============================] - 52s - loss: 0.0047 - actual_accuracy: 0.9894 - val_loss: 0.1722 - val_actual_accuracy: 0.8719


In [19]:
name = '40_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [3]:
dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
feature_dictionary = create_feature_dictionary()
syllable_dictionary = create_syllables_dictionary(content, vowels)
max_syllable = get_max_syllable(syllable_dictionary)
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!


In [8]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)